In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sarp.utils import load_wander_data, separate_train_test, load_transition_data

: 

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)],
        )
    except RuntimeError as e:
        print(e)

UndefVarError: UndefVarError: tf not defined

This notebook provides an example of policy repair using SARP for the robot navigation in hospital. This script assumes that a pre-trained policy and a predictive model are already available. To pre-train a policy for this example run [0_pretrain_policy.py](0_pretrain_policy.py). Also to train a predictive model run [1_pretrain_predictive_model.py](1_pretrain_predictive_model.py). Here are the descriptions of models:
- policy - input: the system state that includes the robot's goal, distancc and heading toward goal, and range sensor readings - output: linear and angular velocities.
- predictive model - input: states and actions - output: collision [0, 1] or no collision [1, 0].